In [2]:
# data normalisation with sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams ["figure.figsize"] = 10,6
import seaborn as sns
from scipy import stats
import math
import datetime
import urllib
import zipfile
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px
import glob
ohe = OneHotEncoder
le = LabelEncoder()
from sklearn.compose import ColumnTransformer
le = LabelEncoder()

from sklearn.model_selection import train_test_split

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression as LR

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import cohen_kappa_score

In [3]:
bike_station = pd.read_csv('stations.csv')


In [4]:
bike_station.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,Georgia & New Hampshire Ave NW,31104,Adams Mill & Columbia Rd NW,W00357,Member
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,10th & U St NW,31503,Florida Ave & R St NW,W00970,Member
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,Adams Mill & Columbia Rd NW,31106,Calvert & Biltmore St NW,W00346,Casual


In [5]:
#FIX THE FRAME

#remove some uneccesary columns, change Duration to minutes, change members to two categories, make the categories workable. 

In [6]:
bike_station2 = bike_station.drop(['Start station', 'End station', 'Bike number'], axis=1)
bike_station2

,Duration,Start date,End date,Start station number,End station number,Member type
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual
...,...,...,...,...,...,...
1226762,300,2011-12-31 23:41:19,2011-12-31 23:46:20,31201,31214,Member
1226763,387,2011-12-31 23:46:43,2011-12-31 23:53:10,31223,31201,Member
1226764,261,2011-12-31 23:47:27,2011-12-31 23:51:49,31107,31602,Member
1226765,2060,2011-12-31 23:55:12,2012-01-01 00:29:33,31205,31222,Member


In [7]:
#most used stations?

In [8]:
bike_station2.dtypes

Duration                 int64
Start date              object
End date                object
Start station number     int64
End station number       int64
Member type             object
dtype: object

In [9]:
bike_station2["Start date"] = bike_station2["Start date"].astype('datetime64[ns]')

In [10]:
bike_station2["End date"] = bike_station2["End date"].astype('datetime64[ns]')

In [11]:
bike_station2["Duration"] = bike_station2["Duration"].astype('int64')

In [12]:
bike_station2.dtypes

Duration                         int64
Start date              datetime64[ns]
End date                datetime64[ns]
Start station number             int64
End station number               int64
Member type                     object
dtype: object

In [13]:
#bike_station2["Start date"]

In [14]:
minute_frame = (bike_station2["Duration"]) / (60)
minute_frame
#lst = []
#for each in minute_frame:
#    lst.append(str(each).split('.')[0])

#final_minute = [int(i) for i in lst]

#final_minute, done shows Duration in minutes! need to change to Int

0          59.133333
1           5.766667
2           9.366667
3           7.233333
4           3.883333
             ...    
1226762     5.000000
1226763     6.450000
1226764     4.350000
1226765    34.333333
1226766     7.800000
Name: Duration, Length: 1226767, dtype: float64

In [15]:
bike_station2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226767 entries, 0 to 1226766
Data columns (total 6 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   Duration              1226767 non-null  int64         
 1   Start date            1226767 non-null  datetime64[ns]
 2   End date              1226767 non-null  datetime64[ns]
 3   Start station number  1226767 non-null  int64         
 4   End station number    1226767 non-null  int64         
 5   Member type           1226767 non-null  object        
dtypes: datetime64[ns](2), int64(3), object(1)
memory usage: 56.2+ MB


In [16]:
bike_station2['dura_min']=(bike_station2['Duration']/60).round(0)

In [17]:
bike_station2.head()

,Duration,Start date,End date,Start station number,End station number,Member type,dura_min
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member,59.0
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual,6.0
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member,9.0
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member,7.0
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual,4.0


In [18]:
bike_station2.dtypes


Duration                         int64
Start date              datetime64[ns]
End date                datetime64[ns]
Start station number             int64
End station number               int64
Member type                     object
dura_min                       float64
dtype: object

In [19]:
minute_frame.dtypes


dtype('float64')

In [20]:
#MEMBERS FIX

In [21]:
bike_station2.loc[bike_station2['Member type'] == "Casual", 'Member cat'] = "1"
bike_station2.loc[bike_station2['Member type'] == "Member", 'Member cat'] = "2"
bike_station2.loc[bike_station2['Member type'] == "Unknown", 'Member cat'] = "1"

In [22]:
bike_station2.head(5)

,Duration,Start date,End date,Start station number,End station number,Member type,dura_min,Member cat
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member,59.0,2
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual,6.0,1
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member,9.0,2
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member,7.0,2
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual,4.0,1


In [23]:
bike_station2["Member cat"].info

<bound method Series.info of 0          2
1          1
2          2
3          2
4          1
          ..
1226762    2
1226763    2
1226764    2
1226765    2
1226766    2
Name: Member cat, Length: 1226767, dtype: object>

In [24]:
#fix the hot encoding

print(ohe)

<class 'sklearn.preprocessing._encoders.OneHotEncoder'>


In [25]:
new_cat = pd.DataFrame(bike_station2, columns=['Member cat'])

new_cat['Member cat'] = new_cat['Member cat'].astype('category')

new_cat['Member cat new'] = new_cat['Member cat'].cat.codes

new_cat

,Member cat,Member cat new
0,2,1
1,1,0
2,2,1
3,2,1
4,1,0
...,...,...
1226762,2,1
1226763,2,1
1226764,2,1
1226765,2,1


In [26]:
#DAY FIX

In [27]:
bike_station3 = bike_station2['Start date'].dt.day_name()

In [28]:
bike_station3

0          Saturday
1          Saturday
2          Saturday
3          Saturday
4          Saturday
             ...   
1226762    Saturday
1226763    Saturday
1226764    Saturday
1226765    Saturday
1226766    Saturday
Name: Start date, Length: 1226767, dtype: object

In [29]:
bike_station4 = bike_station2['End date'].dt.day_name()

In [30]:
bike_station4

0          Saturday
1          Saturday
2          Saturday
3          Saturday
4          Saturday
             ...   
1226762    Saturday
1226763    Saturday
1226764    Saturday
1226765      Sunday
1226766      Sunday
Name: End date, Length: 1226767, dtype: object

In [31]:
bike_station2.head()

,Duration,Start date,End date,Start station number,End station number,Member type,dura_min,Member cat
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member,59.0,2
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual,6.0,1
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member,9.0,2
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member,7.0,2
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual,4.0,1


In [32]:
#final_bike_frame = [bike_station2, bike_station3, bike_station4, final_minute]
# does not look very good....

In [33]:
#merged_df = pd.concat([bike_station2, bike_station3, bike_station4, final_minute])

#TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [34]:
#TESTING AND SUCH

In [37]:
x_data = bike_station2["dura_min", "Start station number"]
y_data = new_cat["Member cat new"]


KeyError: ('dura_min', 'Start station number')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data ,test_size = 0.3, shuffle=False)
x_train, x_test, y_train, y_test

(0         3548
 1          346
 2          562
 3          434
 4          233
           ... 
 858731     785
 858732     495
 858733     701
 858734     180
 858735    1079
 Name: Duration, Length: 858736, dtype: int64,
 858736     1204
 858737     1202
 858738     2580
 858739     1049
 858740      841
            ... 
 1226762     300
 1226763     387
 1226764     261
 1226765    2060
 1226766     468
 Name: Duration, Length: 368031, dtype: int64,
 0         1
 1         0
 2         1
 3         1
 4         0
          ..
 858731    1
 858732    1
 858733    1
 858734    1
 858735    0
 Name: Member cat new, Length: 858736, dtype: int8,
 858736     1
 858737     1
 858738     0
 858739     0
 858740     1
           ..
 1226762    1
 1226763    1
 1226764    1
 1226765    1
 1226766    1
 Name: Member cat new, Length: 368031, dtype: int8)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data,test_size=0.2, random_state=42)

In [ ]:
# data normalisation with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform testing data
X_test_norm = norm.transform(X_test)

ValueError: Expected 2D array, got 1D array instead:
array=[2249.  504.  363. ...  935. 1237.  229.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
#bike_station.isnull().any()

In [ ]:
reg = LR().fit(x_data, y_data)

In [ ]:
reg.score(x_data, y_data)

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[float64]'>)

In [ ]:
y_predicted = reg.predict(x_test)

In [ ]:
reg.coef_

array([-5.25730215e-05,  4.36012701e-05,  5.50321730e-05])

In [ ]:
reg.intercept_

-2.2253248156715673

In [ ]:
reg.predict #(np.array([[3, 5]])) ex.

<bound method LinearModel.predict of LinearRegression()>

In [ ]:
y_predicted = reg.predict(x_test)
y_predicted

array([0.80398807, 0.80409321, 0.71787972, ..., 0.85638507, 0.7451669 ,
       0.8234522 ])

In [ ]:
#Supply and Demand analysis (based on bike stations).

#take the most used start station vs. the most used end station.